# Pika-analyysi Exceliin

Frekvenssitaulukot, ristiintaulukoinnit ja keskeiset tunnusluvut nopeasti Exceliin. Pika-analyysi soveltuu erityisesti kyselytutkimusdatojen analysointiin. Monivalinnalla tarkoitan seuraavassa monivalitakysymyksistä (vastaaja voi valita tarjotuista vaihtoehdoista useampiakin) johdettuja muuttujia. Monivalinnan jokainen vaihtoehto on datassa oma muuttujansa: muuttujan arvo on 1, jos vaihtoehto on valittu; muutoin muuttujan arvo puuttuu (NaN) tai on 0.

Pika-analyysiä voit käyttää seuraavasti:

<ul><li>Vaihda avattavan datan tilalle oma data.</li>
    <li>Määritä muuttujien tyypit listoina (tarpeettomat, kategoriset, mielipideasteikot ja monivalinnat). Jätä lista tyhjäksi (pelkät hakasulkeet), jos kyseinen tyyppi puuttuu datastasi.</li>
<li>Koodin suorittamisen jälkeen Excel-tiedosto pika.xlsx löytyy samasta kansiosta kuin tämä koodi.</li></ul>

Lisätietoa Pythonin käytöstä data-analytiikassa https://tilastoapu.wordpress.com/python/

In [1]:
import pandas as pd

# Vaihda tähän oma data
df = pd.read_excel('http://taanila.fi/data1.xlsx')
df.head()

,nro,sukup,ikä,perhe,koulutus,palveluv,palkka,johto,työtov,työymp,palkkat,työteht,työterv,lomaosa,kuntosa,hieroja
0,1,1,38,1,1.0,22.0,3587,3,3.0,3,3,3,NaN,NaN,NaN,NaN
1,2,1,29,2,2.0,10.0,2963,1,5.0,2,1,3,NaN,NaN,NaN,NaN
2,3,1,30,1,1.0,7.0,1989,3,4.0,1,1,3,1.0,NaN,NaN,NaN
3,4,1,36,2,1.0,14.0,2144,3,3.0,3,3,3,1.0,NaN,NaN,NaN
4,5,1,24,1,2.0,4.0,2183,2,3.0,2,1,2,1.0,NaN,NaN,NaN


In [2]:
# Määritä listat; jätä lista tyhjäksi (pelkät hakasulkeet), jos kyseinen tyyppi puuttuu
tarpeettomat = ['nro']
kategoriset = ['sukup','perhe','koulutus']
mielipideasteikot = ['johto','työtov','työymp','palkkat','työteht']
monivalinnat = ['työterv','lomaosa','kuntosa','hieroja']

In [3]:
# Alustavat toimet

df = df.drop(tarpeettomat, axis=1) # Tarpeettomien poisto

kategoriset2 = kategoriset + mielipideasteikot # Laajennettu lista kategorisista

if kategoriset + monivalinnat:
    df_kvantit = df.drop(kategoriset + monivalinnat, axis=1)
    kvantit = df_kvantit.columns # Lista määrällisistä muuttujista

# Alustan ExcelWriterin kirjoittamaan uuteen tiedostoon kuvailu.xlsx
writer = pd.ExcelWriter('pika.xlsx', engine='xlsxwriter')

# Prosentti-muotoilu, jota käytän frekvenssitaulukoissa ja ristiintaulukoinneissa
format = writer.book.add_format({'num_format': '0.0 %'})


# Frekvenssitaulukot

if kategoriset2:
    # Pidän kirjaa Excelin rivinumerosta rivi-muuttujan avulla
    rivi = 0
    # Käyn for-silmukalla läpi kaikki kategoriset muuttujat
    for var in kategoriset2:
        # Lasken frekvenssit df1-nimiseen dataframeen
        df1 = pd.crosstab(df[var], 'lkm')
        # Lasken df1:een prosentit
        df1['prosenttia'] = df1/df1.sum()
        # Lisään df1:een Yhteensä-rivin
        df1.loc['yhteensä'] = df1.sum()
        # Kirjoitan df1:en Excel-tiedoston Frekvenssit-taulukkovälilehdelle
        df1.to_excel(writer, sheet_name='Frekvenssit', startrow=rivi)
        # Kasvatan rivinumeroa; shape[0] antaa df1:n rivimäärän
        rivi = rivi + df1.shape[0] + 2
    # Lisään C-sarakkeeseen prosenttimuotoilun
    writer.sheets['Frekvenssit'].set_column('C:C', cell_format=format)


# Ristiintaulukoinnit
    
    rivi = 3
    ohje = 'Kaikki ristiintaulukoinnit kategorisille ja mielipideasteikollisille. Prosentit ovat prosentteja sarakkeen summasta (n).'
    for var1 in kategoriset2:
        for var2 in kategoriset2:
            if var1 != var2:
                df1 = pd.crosstab(df[var1], df[var2])
                df2 = pd.crosstab(df[var1], df[var2], normalize='columns')
                df2.index.name=var1+'/'+var2 
                df2.loc['n'] = df1.sum()
                df2.to_excel(writer, sheet_name = 'Ristiintaulukoinnit', startrow = rivi)
                for i in range(rivi+1, rivi+df2.shape[0]):
                    writer.sheets['Ristiintaulukoinnit'].set_row(i, cell_format=format)
                rivi = rivi + df2.shape[0] + 2
    worksheet = writer.sheets['Ristiintaulukoinnit']
    worksheet.write(0, 0, ohje)
            
# Tunnusluvut
            
# Tunnusluvut määrällisille ja mielipideasteikollisille Tunnusluvut-taulukkovälilehdelle
df1 = df_kvantit.describe()
df1.to_excel(writer, sheet_name = 'Tunnusluvut')


# Tunnusluvut kategoristen määräämissä ryhmissä
if kategoriset2:
    rivi = df1.shape[0]+2
    for var1 in kategoriset2:
        for var2 in kvantit:
            if var1 != var2:
                df1 = df.groupby(var1)[var2].describe()
                df1.index.name = var1+'/'+var2
                df1.to_excel(writer, sheet_name = 'Tunnusluvut', startrow = rivi)
                rivi = rivi + df1.shape[0]+2


# Korrelaatiot                
                
# Korrelaatiot Korrelaatiot-taulukkovälilehdelle
df1 = df_kvantit.corr()
df1.to_excel(writer, sheet_name = 'Korrelaatiot')


# Korrelaatiot kategoristen muuttujien määräämissä ryhmissä
if kategoriset2:
    rivi = df1.shape[0]+2
    for var1 in kategoriset2:
        df1 = df.groupby(var1)[kvantit].corr()
        df1.to_excel(writer, sheet_name = 'Korrelaatiot', startrow = rivi)
        rivi = rivi + df1.shape[0]+2

        
# Monivalinnat

ohje = 'Prosentit ovat prosentteja n-arvosta (ryhmän koko).'
# Monivalinnat Monivalinnat-taulukkovälilehdelle
if monivalinnat:
    rivi = 3
    sarake = len(monivalinnat)+2
    for var in monivalinnat:
        df1 = df[monivalinnat].sum().to_frame()
        df1 = df1.rename(columns = {0: 'lkm'})
        df2 = pd.DataFrame()
        df2['% vastaajista'] = df1['lkm']/df.shape[0]
        df2.loc['n','% vastaajista'] = df.shape[0]
        df1.T.to_excel(writer, sheet_name = 'Monivalinnat', startrow=rivi)
        df2.T.to_excel(writer, sheet_name = 'Monivalinnat', startrow=rivi, startcol=sarake)
        writer.sheets['Monivalinnat'].set_column(sarake+1,sarake+len(monivalinnat), cell_format=format)
    
# Monivalinnat kategoristen määräämissä ryhmissä
    rivi = rivi+df1.T.shape[0]+2
    for var1 in kategoriset2:
        df1 = df.groupby(var1)[monivalinnat].sum()
        df1.to_excel(writer, sheet_name = 'Monivalinnat', startrow=rivi)
        for value in df[var1].dropna().unique():
            value_total=df[var1].value_counts()[value]
            df1.loc[value]=df1.loc[value]/value_total
            df1.loc[value, 'n']=value_total
            df1.to_excel(writer, sheet_name = 'Monivalinnat', startrow=rivi, startcol=sarake)
        rivi = rivi+df1.shape[0]+2
    worksheet = writer.sheets['Monivalinnat']
    worksheet.write(0, 0, ohje)

# Excel-tiedoston tallennus
writer.save()